In [1]:
import pymysql.cursors
cnx = pymysql.connect(host='localhost',
                             user='root',
                             password='gzy1589745',
                             db='lahman2017',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [2]:
class RDBTable:
    def __init__(self, table_name, primary_key):
        self.table_name = table_name 
        self.primary_key = primary_key
        
    def DBRunQuery(self,q):
        cursor=cnx.cursor()
        print ("Query = ", q)
        cursor.execute(q);
        r = cursor.fetchall()
        return r

    def templateToWhereClause(self,t):
        s = ""
        for (k,v) in t.items():
            if s != "":
                s += " AND "
            s += k + "='" + v + "'"

        if s != "":
            s = "WHERE " + s;
        return s
    def fieldsToColumName(self,fields):
        if fields == None:
            field = "* "
        else:
            field = ""
            for f in fields:
                field += f + ","
        return field[:-1]
    
    def templateToKeysValues(self,t):
        s1 = "("
        s2 = "("
        for (k,v) in t.items():
            s1 += k + ","
            s2 += "'" + v + "',"
        s1 = s1[:-1] + ")"
        s2 = s2[:-1] + ")"
        s = [s1,s2]
        return s
    
    def primary_key_error(self):
        q = "SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE table_name='" + self.table_name + "'"
        columns = [dic.get('COLUMN_NAME') for dic in self.DBRunQuery(q)]
        for k in self.primary_key:
            if k not in columns:
                return True
        else:
            return False
            
    def invalid_keys_error(self,t):
        q = "SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE table_name='" + self.table_name + "'"
        columns = [dic.get('COLUMN_NAME') for dic in self.DBRunQuery(q)]
        for k in t:
            if k not in columns:
                return True
        else:
            return False
        
    def insert_error(self,t):
        for key in self.primary_key:
            if not t.__contains__(key):
                return "err1"
            table = self.find_by_primary_key([t.get(key)])
            if table != ():
                return "err2"
            
    def find_by_template(self,t,fields = None):
        if self.invalid_keys_error(t):
            return print("Error: find_by_template with invalid keys")
        w = self.templateToWhereClause(t)
        c = self.fieldsToColumName(fields)
        q = "SELECT " + c + " FROM " + self.table_name + " " + w + ";"
        result = self.DBRunQuery(q)
        return result
    
    def find_by_primary_key(self,string,fields = None):
        if self.primary_key_error():
            print("Error: primary keys are not a subset of the columns in the underlying file/table")
        dic = dict(zip(self.primary_key,string))
        w = self.templateToWhereClause(dic)
        c = self.fieldsToColumName(fields)
        q = "SELECT " + c + " FROM " + self.table_name + " " + w + ";"
        result = self.DBRunQuery(q)
        return result
        
    def insert(self,t):
        if self.invalid_keys_error(t):
            return print("Error: insert with invalid keys")
        if self.insert_error(t) == "err1":
            print("Error: insert without primary keys")
        elif self.insert_error(t) == "err2":
            print("Error: insert duplicate primary keys")
        s = self.templateToKeysValues(t)        
        q = "INSERT INTO " + self.table_name + s[0] + " VALUES " + s[1] + ";"        
        self.DBRunQuery(q)
        
    def delete(self,t):
        if self.invalid_keys_error(t):
            return print("delete with invalid keys")
        w = self.templateToWhereClause(t)
        q = "DELETE FROM " + self.table_name + " " + w + ";"
        self.DBRunQuery(q)


In [13]:
def test_template(test_name,names,primary_key, template, fields=None):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Template = ", template)
    print("Fields = ", fields)

    try:
        rdbt = RDBTable(names, primary_key)
        
        r = rdbt.find_by_template(template, fields)
        print("Result table:")
        print(r)
    except ValueError as ve:
        print("Exception = ", ve)

In [14]:
def test_primary_key(test_name,names,primary_key, values, fields=None):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Values = ", values)
    print("Fields = ", fields)

    try:
        rdbt = RDBTable(names, primary_key)

        r = rdbt.find_by_primary_key(values, fields)
        print("Result table:")
        print(r)
    except ValueError as ve:
        print("Exception = ", ve)

In [15]:
def test_insert(test_name,names, primary_key, row, fields=None):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Row to insert = ", row)
    
    try:
        rdbt = RDBTable(names,primary_key)
        
        rdbt.insert(row)
    except ValueError as ve:
        print("Exception = ", ve)


In [16]:
def test_delete(test_name,names,primary_key, row, fields=None):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Row to insert = ", row)
    
    try:
        rdbt = RDBTable(names, primary_key)
        
        rdbt.delete(row)
    except ValueError as ve:
        print("Exception = ", ve)

In [18]:
test_template("Test1", "People",["playerID"],
              {"birthMonth": "9", "nameLast": "Williams"}, ["nameLast", "nameFirst", "birthMonth", "birthYear"])
test_template("Test2", "Batting",["playerID", "yearID", "teamID", "stint"],
              {"playerID": "willite01", "yearID": "1961"}, ["playerID", "yearID", "teamID", "AB", "H", "HR"])




*******************************
Test name =  Test1
Template =  {'birthMonth': '9', 'nameLast': 'Williams'}
Fields =  ['nameLast', 'nameFirst', 'birthMonth', 'birthYear']
Query =  SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE table_name='People'
Query =  SELECT nameLast,nameFirst,birthMonth,birthYear FROM People WHERE birthMonth='9' AND nameLast='Williams';
Result table:
[{'nameLast': 'Williams', 'nameFirst': 'Bernie', 'birthMonth': '9', 'birthYear': '1968'}, {'nameLast': 'Williams', 'nameFirst': 'Don', 'birthMonth': '9', 'birthYear': '1931'}, {'nameLast': 'Williams', 'nameFirst': 'Don', 'birthMonth': '9', 'birthYear': '1935'}, {'nameLast': 'Williams', 'nameFirst': 'Nick', 'birthMonth': '9', 'birthYear': '1993'}, {'nameLast': 'Williams', 'nameFirst': 'Randy', 'birthMonth': '9', 'birthYear': '1975'}, {'nameLast': 'Williams', 'nameFirst': 'Stan', 'birthMonth': '9', 'birthYear': '1936'}]


*******************************
Test name =  Test2
Template =  {'playerID': 'willite01',

In [19]:
test_primary_key("Test1", "People",["playerID"],
              ["willibe02"], ["nameLast", "nameFirst", "birthMonth", "birthYear"])
   
test_primary_key("Test2", "Batting", ["playerID", "yearID", "teamID", "stint"],
              ['willite01','BOS','1960','1'], ["playerID", "yearID", "teamID"])



*******************************
Test name =  Test1
Values =  ['willibe02']
Fields =  ['nameLast', 'nameFirst', 'birthMonth', 'birthYear']
Query =  SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE table_name='People'
Query =  SELECT nameLast,nameFirst,birthMonth,birthYear FROM People WHERE playerID='willibe02';
Result table:
[{'nameLast': 'Williams', 'nameFirst': 'Bernie', 'birthMonth': '9', 'birthYear': '1968'}]


*******************************
Test name =  Test2
Values =  ['willite01', 'BOS', '1960', '1']
Fields =  ['playerID', 'yearID', 'teamID']
Query =  SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE table_name='Batting'
Query =  SELECT playerID,yearID,teamID FROM Batting WHERE playerID='willite01' AND yearID='BOS' AND teamID='1960' AND stint='1';
Result table:
()


In [20]:
test_insert("Insert Test 1", "People",["playerID"],
            {"playerID": "zg2308", "nameLast": "Gao", "nameFirst": "Zheyao"},
            )

test_template("Find insert 1", "People", ["playerID"],
              {"playerID": "zg2308", "nameLast": "Gao", "nameFirst": "Zheyao"}, ["nameLast", "nameFirst", "playerID"]
              )

test_insert("Insert Test 2", "People",["playerID"],
            {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"},
            )
test_template("Find Test 2", "People",["playerID"],
            {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"}, ["nameLast", "nameFirst", "playerID"]
            )



*******************************
Test name =  Insert Test 1
Row to insert =  {'playerID': 'zg2308', 'nameLast': 'Gao', 'nameFirst': 'Zheyao'}
Query =  SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE table_name='People'
Query =  SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE table_name='People'
Query =  SELECT * FROM People WHERE playerID='zg2308';
Query =  SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE table_name='People'
Query =  SELECT * FROM People WHERE playerID='zg2308';
Query =  INSERT INTO People(playerID,nameLast,nameFirst) VALUES ('zg2308','Gao','Zheyao');


*******************************
Test name =  Find insert 1
Template =  {'playerID': 'zg2308', 'nameLast': 'Gao', 'nameFirst': 'Zheyao'}
Fields =  ['nameLast', 'nameFirst', 'playerID']
Query =  SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE table_name='People'
Query =  SELECT nameLast,nameFirst,playerID FROM People WHERE playerID='zg2308' AND nameLast='Gao' AND nameFirst='Zheyao';
Res

In [21]:
test_delete("Delete Test 1", "People", ["playerID"],
            {"playerID": "zg2308", "nameLast": "Gao", "nameFirst": "Zheyao"},
            )

test_template("Find Test 1", "People", ["playerID"],
              {"playerID": "zg2308", "nameLast": "Gao", "nameFirst": "Zheyao"}, ["nameLast", "nameFirst", "playerID"]
              )

test_delete("Delete Test 2", "People", ["playerID"],
            {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"},
            )
test_template("Find Test 2", "People", ["playerID"],
            {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"}, ["nameLast", "nameFirst", "playerID"]
            )



*******************************
Test name =  Delete Test 1
Row to insert =  {'playerID': 'zg2308', 'nameLast': 'Gao', 'nameFirst': 'Zheyao'}
Query =  SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE table_name='People'
Query =  DELETE FROM People WHERE playerID='zg2308' AND nameLast='Gao' AND nameFirst='Zheyao';


*******************************
Test name =  Find Test 1
Template =  {'playerID': 'zg2308', 'nameLast': 'Gao', 'nameFirst': 'Zheyao'}
Fields =  ['nameLast', 'nameFirst', 'playerID']
Query =  SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE table_name='People'
Query =  SELECT nameLast,nameFirst,playerID FROM People WHERE playerID='zg2308' AND nameLast='Gao' AND nameFirst='Zheyao';
Result table:
()


*******************************
Test name =  Delete Test 2
Row to insert =  {'playerID': 'dff1', 'nameLast': 'Ferguson', 'nameFirst': 'Donald'}
Query =  SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE table_name='People'
Query =  DELETE FROM People WHERE 